In [21]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import sympy as sp
import handcalcs.render
import forallpeople as si
import matplotlib.pyplot as plt
%matplotlib inline
from math import tan, sin,cos, pi, radians, sqrt, trunc
from IPython.display import display, Markdown
si.environment('structural', top_level=True)

<h2>KBE Beam verification</h2>

<h3>Compartment: Gym</h3>

<img src = "KBE_beam_E4.png" width=700>

<h4>Loads</h4>

In [22]:
%%render 2

l_influence = (2.58*m)
psi_11 = 0.4
G_k = (4.67+1.21)*kN/m**2*l_influence+3.31*kN/m
Q_k = 5.8*kN/m**2*l_influence


<IPython.core.display.Latex object>

Load combination for fire:

In [23]:
%%render 2
q_d = G_k+psi_11*Q_k

<IPython.core.display.Latex object>

<h4>Beam parameters</h4>

In [24]:
%%render param 2

L = 8.4*m
b = 360*mm
b_top = b-125*mm
h = 470*mm
d_1 = h-50*mm
d_2 = h-100*mm
A_t = 1.42*10**5*mm**2
fck_20 = 45*MPa
phi_s = 12.5*mm
E_s20 = 200*GPa


<IPython.core.display.Latex object>

In [25]:
fyk_20 = 500*MPa

From CONFIRE the following temperatures and parameters were obtained. An example of input (Position I) is shown in the next figure

<img src = "confire-kbe1.png" width=300>


In [26]:
tp = pd.read_excel("confire-kbe.xlsx" , sheet_name = "temp")
tp = tp.set_index("Position")
tp

,"THOT(x,y)","TCOLD(x,y)",xi_sH,xi_sC
Position,,,,
I,503,503,0.1491,0.6109
II,391,405,0.3523,0.7787
III,348,366,0.4363,0.8247
IV,338,354,0.4562,0.8370
V,336,351,0.4596,0.8399
VI,336,351,0.4601,0.8404
VII,212,278,0.6865,0.8989
VIII,144,224,0.8116,0.9322
IX,127,203,0.8411,0.9436


In [27]:
#for each xi_sH create a variable with the name of the xi_sH_index and the value of the xi_sH
for i in tp.index:
    exec("xi_sH_"+str(i)+" = tp.loc[i,'xi_sH']")

for i in tp.index:
    exec("xi_sC_"+str(i)+" = tp.loc[i,'xi_sC']")



And the concrete reduction factors:

<img src = "confire-kbexi.png" width=300>

In [28]:
%%render 2
xi_CMH = 1
xi_CMC = 0.9680
eta_H = 0.9159
eta_C = 0.8161

<IPython.core.display.Latex object>

**Before fire**

Using effective prestress strength per wire = 110 kN, provided by the supplier

In [29]:
%%render 2

M_max_ALS = q_d*L**2/8
F_su20 = (110*kN)
y_20 = 11*F_su20/(fck_20*b_top)

<IPython.core.display.Latex object>

In [30]:
%%render long 2

F_1 = 6*F_su20
F_2 = 5*F_su20
M_u20 = F_1*(d_1-y_20/2)+F_2*(d_2-y_20/2)
mu_20 = M_max_ALS/M_u20

<IPython.core.display.Latex object>

**HOT - During fire**

In [31]:
%%render symbolic
F_suH = (xi_sH_I+xi_sH_II+xi_sH_III+xi_sH_IV+xi_sH_V+xi_sH_VI+xi_sH_VII+xi_sH_VIII+xi_sH_IX+xi_sH_X+xi_sH_XI)*F_su20

<IPython.core.display.Latex object>

In [32]:
%%render
F_suH

<IPython.core.display.Latex object>

In [33]:
%%render 2

F_H1 = (xi_sH_I+xi_sH_II+xi_sH_III+xi_sH_IV+xi_sH_V+xi_sH_VI)*F_su20
F_H2 = (xi_sH_VII+xi_sH_VIII+xi_sH_IX+xi_sH_X+xi_sH_XI)*F_su20

<IPython.core.display.Latex object>

In [34]:
%%render 2
y_H = F_suH/(xi_CMH*fck_20*b_top*eta_H)
M_u_H = F_H1*(d_1-0.5*y_H)+F_H2*(d_2-0.5*y_H)
mu_H = M_max_ALS/M_u_H

<IPython.core.display.Latex object>

**COLD - After fire**

In [35]:
%%render symbolic
F_suC = (xi_sC_I+xi_sC_II+xi_sC_III+xi_sC_IV+xi_sC_V+xi_sC_VI+xi_sC_VII+xi_sC_VIII+xi_sC_IX+xi_sC_X+xi_sC_XI)*F_su20


<IPython.core.display.Latex object>

In [36]:
%%render 2
y_C = F_suC/(xi_CMC*fck_20*b_top*eta_C)

<IPython.core.display.Latex object>

In [37]:
if y_C<d_2:
    display(Markdown('Compression zone lower than slab thickness (270 mm), only $b_{top}$ considered'))
else:
    display(Markdown('Compression zone higher than slab thickness (270 mm), lower width must be considered'))

Compression zone lower than slab thickness (270 mm), only $b_{top}$ considered

In [38]:
%%render long 2
F_C1 = (xi_sC_I+xi_sC_II+xi_sC_III+xi_sC_IV+xi_sC_V+xi_sC_VI)*F_su20
F_C2 = (xi_sC_VII+xi_sC_VIII+xi_sC_IX+xi_sC_X+xi_sC_XI)*F_su20
M_u_C = F_C1*(d_1-0.5*y_C)+F_C2*(d_2-0.5*y_H)
mu_C = M_max_ALS/M_u_C

<IPython.core.display.Latex object>

Over-reinforcment check

In [39]:
%%render 2

varepsilon_s = (d_1-5/4*y_C)/(5/4*y_C)*0.35/100/xi_CMC
varepsilon_smin = fyk_20/E_s20 + 0.2/100

<IPython.core.display.Latex object>

In [40]:
if varepsilon_s>varepsilon_smin and mu_H<1 and mu_C<1:
    display(Markdown('The beam is verified for fire'))
else:
    display(Markdown('The beam is not verified for fire'))

The beam is verified for fire